## 17.1:  monte python

We start this notebook with a simple game:  heads I win, tails you lose.

What values do you get when you run this cell again and again?

In [ ]:
import random
prn = random.random()    # uniform pseudorandom number (prn) in interval [0.0, 1.0)
print(prn)

Now run this cell again and again.  What <u>value</u> do you get?

In [ ]:
import random
random.seed(12345678)    # specify a position in the random number sequence to start
prn = random.random()    # uniform prn [0.0, 1.0)
print(prn)

### 17.1.1:  a simple random number generator

<i>Linear Congruential Generators</i> are very simple pseudorandom number generators to implement.  The classic is the Park & Miller generator which has some reasonably good statistical properties of randomness.  It is implemented here to list ten pseudorandom numbers.  Note how the seed is updated by each successive random integer produced by the generator.

In [ ]:
multiplier =       48271
modulus    =  2147483647
seed       = 12345678910

def lcg(iprn):
    """pseudorandom number generator based on Park & Miller revised algorithm."""
    iprn = (multiplier * iprn) % modulus
    return iprn

for i in range(10):
    iprn = lcg(seed)
    prn = iprn / modulus
    print(prn)
    seed = iprn

However, all linear congruential generators have a fundamental flaw.

Below is an even simpler generator to clearly see what the problem is.  When linear congruential generators are used to specify 2D coordinates, the coordinates all fall along specific lines.  Or, in 3D, along planes.  It is impossible to generate "random" coordinates not on these lines (or not in the planes), making any assertion of randomness highly questionable.

In [ ]:
import numpy as np
def bad_lcg(iprn):
    '''a really crummy linear congruential
       pseudorandom number generator'''
    iprn = (17 * iprn) % 113
    return iprn
seed = 97
iprn = bad_lcg(seed)

xy = np.zeros([100,2])
for i in range(100):
    iprn = bad_lcg(iprn) ; prn = iprn/113 ; xy[i,0] = prn
    iprn = bad_lcg(iprn) ; prn = iprn/113 ; xy[i,1] = prn
#    print("x, y =", xy[i,0], xy[i,1])  # cycle is 54

import matplotlib.pyplot as plt
plt.figure(1)
plt.plot(xy[:,0], xy[:,1], 'ob', lw=2)
plt.xlabel('x' , fontsize=12)
plt.ylabel('y', fontsize=12)
plt.show()
print("Random numbers fall mainly in the planes")
print("   -- George Marsaglia")

### 17.1.2:  a Monte Carlo simulation to demonstrate the exercise in Preliminaries Section 17.2

You may want to try the "guess 25 random digits" game in Preliminaries Section 17.2 before you look at and run this code.

In [ ]:
import random
import numpy as np

nsims = 10000  # number of simulations to run
n = 25         # number of digits to sample in each simulation

array = np.zeros(n)   # array of n random digits
tally = 0             # tally counts the number of samples of size n with at least one double

for j in range(nsims):
    array[0] = random.randint(0, 9)        # get the first digit
    for i in range(1, n):                  # loop over remaining digits to sample
        array[i] = random.randint(0, 9)
        # if there are there two of the same digit in a row and ignore the possibility
        # that it is a triple, quadruple, etc., or that there could be more doubles in
        # the array.  The presence of at least two in a row triggers a tally.
        if (array[i] == array[i-1]):
            tally += 1
            break        # double detected; go to next simulation
frac = tally/nsims       # convert to fraction
print("Fraction of sequences of", n, "digits with at least one double (as %) = ", 100.0*frac)

### 17.1.3:  flipping a coin, again and again and again and...

So computers generating pseudorandom numbers (using a decent algorithm) may do better than humans, but will it always be fair?  Ok, but will it always <u>feel</u> fair?

In [ ]:
"""A fair (?) game -- tossing a coin"""

import random
random.seed(20200407)

n_trials = 10
n_heads  = 0
for i in range(n_trials):
    prn = random.random()    # uniform [0, 1.0)
    if (prn < 0.5):
        n_heads += 1
win = 100 * (n_heads / n_trials)
print("winnings = " + str(win) + "%")

Are you satisfied that you only won 30% of the time?  Does that seem fair?

To reconcile that you need to take or recall MAU -- ENGR-2600 Modeling & Analysis of Uncertainty.  In particular, <b><i>the law of large numbers:</i></b>  "the average of results from a <u>large number</u> of trials should be <u>close</u> to the expected value, and will <u>tend</u> to become closer as more trials are performed.”

In [ ]:
import random
random.seed(20200407)
for n_trials in [10, 100, 1000, 10000, 100000, 1000000]:
    n_heads  = 0
    for i in range(n_trials):
        prn = random.random()    # uniform [0, 1.0)
        if (prn < 0.5):
            n_heads += 1
    win = 100 * (n_heads / n_trials)
    print("winnings = " + str(win) + "%")

Run the code below to observe how numerous flips of a coin tends toward 50% heads.  But then try different seeds and different numbers of sumulations and question how well anything converges when it is random.

In [ ]:
from math import pi
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(20200407)    # try the 6th and the 8th as well

n_total  = 2000          # 2000 simulations looks nice; 3000 actually less so
darts = np.zeros(n_total)
pie   = np.zeros(n_total)

tally = 0
for n in range(n_total):            # loop over darts thrown
    x = 2.0*random.random() - 1.0   # scale x coordinate to [-1.0, 1.0)
    y = 2.0*random.random() - 1.0   # scale y coordinate to [-1.0, 1.0)
    if (x**2 + y**2 <= 1.0):        # is the random coordinate inside the circle?
        tally += 1
    darts[n] = float(n+1)           # start with 1st dart, not 0th dart
    pie[n]   = 4.0 * tally / darts[n]

plt.figure(1, figsize=(10, 4))                   # wide figure
plt.xlim(0.0, float(n_total))                    # restrict x domain to 0, n_total
plt.ylim(2.4, 4.0)                               # restrict y range to around pi = 3.14
plt.plot([0.0, float(n_total)], [pi, pi], '-b')  # line [x1, x2], [y1, y2] showing pi
plt.plot(darts, pie, '-r', lw=2)                 # red line is evolution of darts thrown
plt.xlabel('number of darts' , fontsize=12)
plt.ylabel('estimate of pi', fontsize=12)
plt.gca().grid()
plt.show()